# Projet de Machine Learning

In [118]:
from tools import *
import math as math
import copy
%matplotlib inline

In [119]:
def gen_arti(centerx=1,centery=1,sigma=0.1,nbex=50,data_type=0,eps=0.02):
    """ Generateur de donnees,
        :param centerx: centre des gaussiennes
        :param centery:
        :param sigma: des gaussiennes
        :param nbex: nombre d'exemples
        :param data_type: 0: melange 2 gaussiennes, 1: melange 4 gaussiennes, 2:echequier
        :param eps: bruit dans les donnees
        :return: data matrice 2d des donnnes,y etiquette des donnnees
    """
    if data_type==0:
         #melange de 2 gaussiennes
         xpos=np.random.multivariate_normal([centerx,centery],np.diag([sigma,sigma]),nbex//2)
         xneg=np.random.multivariate_normal([-centerx,-centery],np.diag([sigma,sigma]),nbex//2)
         data=np.vstack((xpos,xneg))
         y=np.hstack((np.ones(nbex//2),-np.ones(nbex//2)))
    if data_type==1:
        #melange de 4 gaussiennes
        xpos=np.vstack((np.random.multivariate_normal([centerx,centery],np.diag([sigma,sigma]),nbex//4),
                        np.random.multivariate_normal([-centerx,-centery],np.diag([sigma,sigma]),nbex//4)))
        xneg=np.vstack((np.random.multivariate_normal([-centerx,centery],np.diag([sigma,sigma]),nbex//4),
                        np.random.multivariate_normal([centerx,-centery],np.diag([sigma,sigma]),nbex//4)))
        data=np.vstack((xpos,xneg))
        y=np.hstack((np.ones(nbex//2),-np.ones(nbex//2)))

    if data_type==2:
        #echiquier
        data=np.reshape(np.random.uniform(-4,4,2*nbex),(nbex,2))
        y=np.ceil(data[:,0])+np.ceil(data[:,1])
        y=2*(y % 2)-1
    # un peu de bruit
    data[:,0]+=np.random.normal(0,eps,nbex)
    data[:,1]+=np.random.normal(0,eps,nbex)
    # on mélange les données
    idx = np.random.permutation((range(y.size)))
    data=data[idx,:]
    y=y[idx]
    return data,y

    

In [116]:
def f_K(x, e, sigma=1):
    return math.exp(-pow(np.linalg.norm(x-e), 2)/(2*pow(sigma,2)))

# Calcul de tous les H(xl) pour le a minimal 
def HA(a, S, dataset, y):
    N = dataset.shape[0]
    Kql= np.zeros((N,N))
    Kal = np.zeros((N,N))
    f = np.zeros((0, 0))
    
    #print "Calcul des fl..."
    for l,x in enumerate(dataset):
        fl = 0
        for j,e in enumerate(S):
            fl += a[j]*f_K(x,e)
        f = np.append(f, fl)
        
    #print "Done"
    for i,x in enumerate(S):
        for j,e in enumerate(S):
            Kql[i,j] = f_K(x,e)
            
    Kql = np.asmatrix(np.asarray(Kql))
    f = np.asmatrix(f.reshape((f.shape[0], 1)))
    y = np.asmatrix(y)

    
    #print "Calcul de H(xl)..."
    return -sum(y.T*f - np.log(1+np.exp(f)))+lamda/2 * a.T * Kql * a

# Calcul le a minimal approximé par Newton-R
def H(x, S, a_prec, dataset, y):

    a = a_optim(a_prec) 
    return HA(a, S, dataset, y), a

def a_optim(a_prec):

    f = K*a_prec
    p = np.multiply(np.asarray(np.exp(f)),1/np.asarray((1+np.exp(f))))
    d = np.asarray(p*(1-p))
    W = np.asmatrix(np.zeros((d.shape[0], d.shape[0])))
    
    for i,element in enumerate(d):
        W[i,i] = element
        
    z = K*a_prec+np.linalg.inv(W)*(y-p)
    return np.linalg.inv(K.T*W*K + lamda*K) * K.T*W*z 

def ivm(dataset, y):

    N = dataset.shape[0]
    a = np.zeros((N,1))
    R = np.zeros((N,dataset.shape[1]))
    S = []
    lamda = 3
    a_s = None
    
    Hk = 1
    Hk_1 = 0
    for i, e in enumerate(dataset):
        R[i] = e

    while abs(Hk-Hk_1)/abs(Hk) > 0.01:
        if R.shape[0] == 0:
            break
            
        #print "Calcul des nouveaux H(xl)..."
        h = np.ones((R.shape[0], 1))
        S_b = []
        for e in S:
            S_b.append(e)
            
        A = []
        for i, xl in enumerate(R):
            S_b.append(xl)
            h[i], a = H(xl, S_b, a, dataset, y)
            A.append(a)
            S_b.pop()
            
            
        #print "Done"
        #print "On recupere le plus petit H(xl)"
        xl_opt = np.argmin(h)
        Hk_1 = Hk
        Hk = h[xl_opt]
        #print h
        #print "Le plus petit est ", Hk, " il est à l'indice : ", xl_opt

        #print "On l'ajoute a S..."

        S.append(R[xl_opt])
        #print "--------------> S:"

        #print "OK"
        #print "On l'enleve a R..."

        data = []
        for i, e in enumerate(R):
            if i == xl_opt:
                pass
            else:
                data.append(e)
        R = np.asarray(data)
        # update de a
        print "Updating a..."

        a = a_optim(A[xl_opt])
        #print "On garde celui associe au meilleur:", A[xl_opt]
        #print "OK"
        print "ecart", abs(Hk-Hk_1)/abs(Hk)
        
    print "S final"
    for el in S:
        print el


In [117]:
data, y = gen_arti()
data = np.asarray(data)
print data
N = data.shape[0]
K = np.asmatrix(np.zeros((N,N)))
lamda=1
for i,x in enumerate(data):
    for j,e in enumerate(data):
        K[i,j] = f_K(x,e)

y = y.reshape((y.shape[0], 1))
ivm(data, y)

[[-0.8903816  -1.27533421]
 [-1.12429597 -0.80478545]
 [ 1.15932768  0.72939264]
 [ 1.03913424  0.95366102]
 [ 1.16474237  1.14003483]
 [-1.25045363 -0.82783133]
 [-1.26494585 -0.77068662]
 [ 1.0326769   0.30938284]
 [ 0.91775241  1.26379024]
 [ 1.99598804  1.65153485]
 [ 0.60375932  0.86172317]
 [-0.67793207 -1.26720217]
 [-0.73399415 -1.26482496]
 [ 0.48105399  1.23856052]
 [-1.02586296 -1.48689578]
 [-1.00111415 -1.08222973]
 [-1.15328151 -0.84802612]
 [-1.12493566 -0.98295361]
 [-0.56293618 -0.62734524]
 [ 1.13153703  0.52816426]
 [ 1.05312344  1.23872194]
 [ 0.67681538  0.69162471]
 [ 0.49850302  1.61147933]
 [ 0.730507    0.97718651]
 [-0.9140822  -1.27567919]
 [-0.95382758 -1.5846839 ]
 [-1.52437161 -1.34779029]
 [-1.18308918 -0.88668944]
 [ 0.74842259  0.91850464]
 [-1.61920211 -1.30258197]
 [ 0.96094619  0.82185578]
 [-0.94738243 -0.88132317]
 [-0.52456338 -1.41889623]
 [-1.37439234 -1.04593406]
 [ 1.26275228  0.85678387]
 [ 1.68857996  0.76976634]
 [-0.81131805 -0.86928082]
 

KeyboardInterrupt: 